In [43]:
import pandas as pd
import funzioni as fx
import numpy as np
import yfinance as yf
from tensorflow.keras.models import load_model
from tensorflow_addons.metrics import F1Score
from multiprocessing import Pool, cpu_count, Manager, Value, Queue
import tensorflow as tf
from datetime import datetime, timedelta, date
import json
import os
import ctypes
from functools import partial
from typing import Tuple

class Trade:
    def __init__(self, nome_ticker, data_inizio, data_fine, importo_da_investire, probabilità_per_acquisto, stop_loss, take_profit, giorni_max_posizione) -> None:
        self.SIMBOLO = nome_ticker
        self.DATA_INIZIO = data_inizio
        self.DATA_FINE = data_fine
        self.BILANCIO_INIZIALE = importo_da_investire
        self.PROBABILITA_PER_ACQUISTO = probabilità_per_acquisto
        self.SL = stop_loss
        self.TP = take_profit
        self.GIORNI_POS = giorni_max_posizione
        self.log = None
        self.model = None
        self.valutazione_modello = None
        self.posizioni = None
        self.bilancio = None
        self.esito = None
        
    def avvia_trading(self, verbose=0) -> None:
        self.log = "\033[42m" + self.SIMBOLO + "\033[0m" + "\n"
        ticker = yf.download(self.SIMBOLO, start=self.DATA_INIZIO, end=self.DATA_FINE, progress=False)
        ticker.index = ticker.index.date
        str = f"Calcolo indicatori ticker {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"
        ticker = fx.crea_indicatori(ticker)
        ticker = fx.imposta_target(ticker)
        ticker.dropna(axis=0, inplace=True)

        str = f"Definizione features e target {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"        
        idx, X, Y, _ = fx.to_XY(ticker, fx.features_prezzo, fx.features_da_scalare_singolarmente, fx.features_meno_piu, fx.features_candele, fx.features_no_scala, fx.elenco_targets, fx.n_timesteps, fx.giorni_previsione, bilanciamento=0)
        str = f"Previsione {self.SIMBOLO}"
        if verbose == 1: print(str)
        self.log += str + "\n"        
        f1_score = F1Score(num_classes=1, average='macro', threshold=self.PROBABILITA_PER_ACQUISTO)
        self.model = load_model("model.h5", custom_objects={'Addons>F1Score': f1_score})
        y_pred = self.model.predict(X)
        self.valutazione_modello = self.model.evaluate(X, Y)

        X = X[:, -1, :]

        df_X = ticker.loc[ticker.index.intersection(idx)]
        y_pred = y_pred.reshape(-1,)
        df_y_pred = pd.DataFrame(y_pred, columns=['prob_True'], index=idx)
        self.dati = pd.concat([df_X, df_y_pred], axis=1)

        self.bilancio = self.BILANCIO_INIZIALE
        pos_aperta = False
        tp = sl = n_azioni = prezzo_acquisto = prezzo_tot = 0
        giorni_posizione = 0
        self.posizioni = pd.DataFrame(columns=["Bilancio", "Direzione", "Prezzo_un", "n_azioni", "Prezzo_tot", "TP", "SL", "Open", "High", "Low", "Esito", "P_L"], index=self.dati.index)

        for idx, row in self.dati.iterrows():
            if self.bilancio <= 0:
                break
            if pos_aperta == False:
                if row["prob_True"] > self.PROBABILITA_PER_ACQUISTO and self.bilancio > 0:
                    #COMPRA
                    prezzo_acquisto = row["Open"]
                    n_azioni = self.bilancio // prezzo_acquisto
                    prezzo_tot = n_azioni * prezzo_acquisto  
                    self.bilancio -= prezzo_tot          
                    tp = prezzo_acquisto * (1 + self.TP)
                    sl = prezzo_acquisto * (1 - self.SL)
                    pos_aperta = True
                    giorni_posizione = 0
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "COMPRA", "Prezzo_un": prezzo_acquisto, "n_azioni": n_azioni, "Prezzo_tot": -prezzo_tot, "TP": tp, "SL": sl, "Open": row["Open"], "High": row["High"], "Low": row["Low"],}
            else:
                if giorni_posizione == self.GIORNI_POS:
                    prezzo_tot_vendita = row['Close'] * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    if prezzo_tot_vendita > prezzo_tot:
                        esito = 'VINCITA'
                    else:
                        esito = 'PERDITA'
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": row['Close'], "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": esito, "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                if row["High"] >= tp:
                    prezzo_tot_vendita = tp * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    giorni_posizione = 0
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": tp, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "VINCITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                elif row["Low"] <= sl:
                    prezzo_tot_vendita = sl * n_azioni
                    self.bilancio += prezzo_tot_vendita
                    pos_aperta = False
                    giorni_posizione = 0
                    pl = round(fx.pct_change(prezzo_tot, prezzo_tot_vendita), 0)
                    self.posizioni.loc[idx] = {"Bilancio": self.bilancio, "Direzione": "VENDI", "Prezzo_un": sl, "n_azioni": n_azioni, "Prezzo_tot": prezzo_tot, "Esito": "PERDITA", "Open": row["Open"], "High": row["High"], "Low": row["Low"], "P_L": pl}
                else:
                    giorni_posizione += 1

        if pos_aperta: 
            self.bilancio += prezzo_tot

        tot_vincite = self.posizioni.loc[self.posizioni["Esito"] == "VINCITA", "Esito"].count()
        tot_perdite = self.posizioni.loc[self.posizioni["Esito"] == "PERDITA", "Esito"].count()
        max_vincita_value = self.posizioni.loc[self.posizioni["Esito"] == "VINCITA", "P_L"].max(skipna=True)
        max_perdita_value = self.posizioni.loc[self.posizioni["Esito"] == "PERDITA", "P_L"].min(skipna=True)
        max_vincita = int(max_vincita_value if max_vincita_value == max_vincita_value else 0)  # max_vincita_value == max_vincita_value è un modo per controllare se non è NaN
        max_perdita = int(max_perdita_value if max_perdita_value == max_perdita_value else 0)
        tot = int(fx.pct_change(self.BILANCIO_INIZIALE, self.bilancio))
        self.esito = (
            f"\nBilancio {self.SIMBOLO} = {round(self.bilancio, 2)}" + "\n" +
            f'Vincite: {tot_vincite}, vincita max: {max_vincita} %' + "\n" + 
            f'Perdite: {tot_perdite}, perdita max: {max_perdita} %' + "\n" +
            f'Totale: {tot} %'
        )
        if verbose == 1: print(self.esito)
        self.log += self.esito + "\n"

class Borsa:
    def __init__(self, n_simboli_contemporanei, bilancio_iniziale, probabilità_per_acquisto, stop_loss, take_profit, giorni_max_posizione, data_inizio=pd.Timestamp(year=2005, month=1, day=1), data_fine=pd.Timestamp.now().normalize()):
        self.N_SIMBOLI = n_simboli_contemporanei

        self.DATA_INIZIO = pd.to_datetime(data_inizio)
        while self.DATA_INIZIO.dayofweek >= 5:
            self.DATA_INIZIO += pd.Timedelta(days=1)

        self.DATA_FINE = pd.to_datetime(data_fine)
        while self.DATA_FINE.dayofweek >= 5:
            self.DATA_FINE -= pd.Timedelta(days=1)

        self.BILANCIO_INIZIALE = bilancio_iniziale
        self.PROBABILITA_PER_ACQUISTO = probabilità_per_acquisto
        self.SL = stop_loss
        self.TP = take_profit
        self.GIORNI_POS = giorni_max_posizione
        self.log = None
        self.f1_score = F1Score(num_classes=1, average='macro', threshold=self.PROBABILITA_PER_ACQUISTO)
        self.model = load_model("model.h5", custom_objects={'Addons>F1Score': self.f1_score})
        self.posizioni = []
        self.bilancio = self.BILANCIO_INIZIALE
        self.lista_tickers = pd.read_parquet("lista_ticker.parquet")
        self.screener = pd.DataFrame(columns=["Ticker", "Prediction"])
    
        try:
            if os.path.exists(f'tickers/_indice.json'):
                with open(f'tickers/_indice.json', 'r') as jsonfile:
                    indice = json.load(jsonfile)
                prima_data = pd.to_datetime(indice['prima_data'])
                ultima_data = pd.to_datetime(indice['ultima_data'])
                
                if (self.DATA_INIZIO < prima_data):
                    scarica_prima = True
                else:
                    scarica_prima = False

                if (self.DATA_FINE > ultima_data):
                    scarica_dopo = True
                else:
                    scarica_dopo = False

                self.scarica_tickers(scarica_prima, scarica_dopo, self.DATA_INIZIO, self.DATA_FINE, append=True)
                self.avvia_screener(scarica_prima, scarica_dopo, self.DATA_INIZIO, self.DATA_FINE, append=True)    
            else:
                self.scarica_tickers(scarica_prima=True, scarica_dopo=True, data_inizio=self.DATA_INIZIO, data_fine=self.DATA_FINE, append=False)
                self.avvia_screener(scarica_prima=True, scarica_dopo=True, data_inizio=self.DATA_INIZIO, data_fine=self.DATA_FINE, append=False)    
        except Exception as e:
            print(str(e))
            
    def inizializza_gpu(self):
        # Ottiene la lista di tutte le GPU fisiche disponibili
        gpus = tf.config.experimental.list_physical_devices('GPU')
        if gpus:
            try:
                # Imposta TensorFlow per utilizzare tutte le GPU visibili
                tf.config.experimental.set_visible_devices(gpus, 'GPU')
                for gpu in gpus:
                    # Imposta TensorFlow per allocare dinamicamente la memoria su ogni GPU disponibile
                    tf.config.experimental.set_memory_growth(gpu, True)
            except RuntimeError as e:
                # L'eccezione viene sollevata se si tenta di impostare la visibilità delle GPU o la crescita della memoria
                # dopo che la sessione di TensorFlow è stata inizializzata
                print(e)

    def scarica_tickers(self, scarica_prima=True, scarica_dopo=True, data_inizio=pd.Timestamp(year=2005, month=1, day=1), data_fine=pd.Timestamp.now().normalize(), append=False) -> None: 
        if scarica_prima or scarica_dopo:
            tot_tickers = len(self.lista_tickers)
            totale_processati = Value('i', 0)
            with Pool(cpu_count()) as p:
                callback_with_args = partial(_callback_tickers, totale_processati=totale_processati, tot_tickers=tot_tickers)
                for i in range(0, tot_tickers):
                    nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                    param = (nome_simbolo, scarica_prima, scarica_dopo, data_inizio, data_fine, append)
                    p.apply_async(_scarica, args=param, callback=callback_with_args)
                p.close()
                p.join()     

        indice = {
            'prima_data': self.DATA_INIZIO.strftime('%Y-%m-%d'),
            'ultima_data': self.DATA_FINE.strftime('%Y-%m-%d')
        }
        with open(f'tickers/_indice.json', 'w') as jsonfile:
            json.dump(indice, jsonfile, indent=4)    
 
    def avvia_screener(self, scarica_prima=True, scarica_dopo=True, data_inizio=pd.Timestamp(year=2005, month=1, day=1), data_fine=pd.Timestamp.now().normalize(), append=False) -> None:
        try:
            tot_tickers = len(self.lista_tickers)
            totale_processati = Value('i', 0)
            coda = Queue()

            with Pool(cpu_count()) as p:
                for i in range(0, tot_tickers):
                    nome_simbolo = self.lista_tickers.iloc[i]["Ticker"]
                    param = (nome_simbolo, scarica_prima, scarica_dopo, data_inizio, data_fine, append, self.PROBABILITA_PER_ACQUISTO, coda)
                    p.apply_async(_screener_scarica, args=param)
                p.close()
                p.join()

            # Qui esegui le predizioni in sequenza
            while not coda.empty():
                nome_simbolo, idx, X, Y = coda.get()
                print(nome_simbolo)
                if X is not None and Y is not None:
                    pred = self.model.predict(X)
                    # Salva o elabora i risultati della previsione
                    scr_temp = pd.DataFrame({'Previsione': pred.flatten().round(2), 'Reale': Y.flatten()}, index=idx)
                    scr_temp.to_hdf(f'screeners/{nome_simbolo}.h5', key='screener', mode='w')
        except Exception as e:
            print(str(e))

def _screener_scarica(nome_simbolo, scarica_prima, scarica_dopo, data_inizio, data_fine, append, probabilità_per_acquisto, coda):
    print("ok")
    try:
        ticker = pd.read_hdf(f'tickers/{nome_simbolo}.h5', 'ticker')
        scr = pd.read_hdf(f'screeners/{nome_simbolo}.h5', 'screener')
        ti_min = scr.index.min()
        ti_max = scr.index.max()
        if append == True:
            if scarica_prima:
                ticker_inizio = ticker[ticker.index < ti_min]
                idx, X, Y, _ = fx.to_XY(ticker_inizio, bilanciamento=0)
                coda.put((nome_simbolo, idx, X, Y))
            if scarica_dopo:
                ticker_fine = ticker[ticker.index > ti_max]
                idx, X, Y, _ = fx.to_XY(ticker_fine, bilanciamento=0)
                coda.put((nome_simbolo, idx, X, Y))
        else:
            idx, X, Y, _ = fx.to_XY(ticker, bilanciamento=0)
            coda.put((nome_simbolo, idx, X, Y))
        scr.to_hdf(f'screeners/{nome_simbolo}.h5', key='screener', mode='w')         
    except Exception as e:
        coda.put((nome_simbolo, None, None, None))
        print(f"Errore durante la preparazione per {nome_simbolo}: {e}")

def _scarica(nome_simbolo, scarica_prima, scarica_dopo, data_inizio, data_fine, append):
    try:
        if append == True:
            ticker = pd.read_hdf(f'tickers/{nome_simbolo}.h5', "ticker")
            ti_min = ticker.index.min()
            ti_max = ticker.index.max()
            if scarica_prima:
                inizio = data_inizio - pd.Timedelta(days=365)
                fine = ti_min - pd.Timedelta(days=1)
                df_inizio = fx.analizza_ticker(nome_simbolo, start=inizio, end=fine, progress=False, dropna_iniziali=True, dropna_finali=False)
                ticker = pd.concat([df_inizio, ticker], axis=0, ignore_index=False)
            if scarica_dopo:
                inizio = ti_max - pd.Timedelta(days=365) 
                fine = data_fine
                df_fine = fx.analizza_ticker(nome_simbolo, start=inizio, end=fine, progress=False, dropna_iniziali=True, dropna_finali=False)
                ticker = ticker[ticker.index < df_fine.index.min()]
                ticker = pd.concat([ticker, df_fine], axis=0, ignore_index=False)
        else:
            ticker = fx.analizza_ticker(nome_simbolo, start=data_inizio, end=data_fine, progress=False, dropna_iniziali=True, dropna_finali=False)
        return nome_simbolo, ticker, ""
    except Exception as e:
        return nome_simbolo, None, str(e)

def _callback_tickers(result, totale_processati, tot_tickers):
    nome_simbolo, ticker, error = result
    if error == "":
        ticker.to_hdf(f'tickers/{nome_simbolo}.h5', key='ticker', mode='w')
    else:
        print(f"Errore su funzione di callback per {nome_simbolo}: {error}")

    with totale_processati.get_lock(): 
        totale_processati.value += 1
    print(f"{totale_processati.value}/{tot_tickers}) Completato ticker {nome_simbolo}")   
 



In [44]:
if __name__ == "__main__":
    TP = 1
    SL = 0.01
    PROBABILITA_PER_ACQUISTO = 0.5
    BILANCIO_INIZIALE = 1000
    DATA_INIZIO = '2005-01-03'
    DATA_FINE = '2023-12-31'
    GIORNI_MAX_POSIZIONE = 20
    N_SIMBOLI = 10

    borsa = Borsa(N_SIMBOLI, BILANCIO_INIZIALE, PROBABILITA_PER_ACQUISTO, SL, TP, GIORNI_MAX_POSIZIONE)

    

In [45]:
borsa.avvia_screener(scarica_prima=True, scarica_dopo=True, data_inizio=pd.Timestamp(2005, 1, 3), data_fine=pd.Timestamp(2023, 11, 14), append=False) 